# Scraper Optimization

In [20]:
from get_urls import *

In [21]:
# tune with line_profiler
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [22]:
# test with EPL 2019-2020
urls = scoresfixtures("https://fbref.com/en/comps/9/3232/schedule/2019-2020-Premier-League-Scores-and-Fixtures", "div_sched_3232_1")

## Original Version

In [23]:
def get_df(urls):
    '''
    Description: This function goes to de URL of the match and treat all data in order to append it in one single Dataframe.
    
    Input:
        - urls: urls of all the match reports
        - league: league
        - season: season
    Output:
        - Dataframe treated from the match saved on my machine excel file
    
    '''
    df_all = pd.DataFrame()
    ind = 0
    
    for url in urls:    
        # make the request
        pg = 'https://fbref.com'
        url_pg = pg+ url
        req = requests.get(url_pg)
        if req.status_code == 200:
            content = req.content
        # accessing data from site
        soup = BeautifulSoup(content, 'html.parser')

        
        table_geral = soup.find_all(class_ = "table_wrapper tabbed")
        table_1 = table_geral[0]
        table_2 = table_geral[1]
        table_3 = soup.find(class_='venuetime')

        
        # possession
        percentage = str(soup.find_all('strong')).split('%')
        possession_1 = int(percentage[0][-2:])
        possession_2 = int(percentage[1][-2:])
        
        # save percentage
        save = str(soup.find_all('td')).split("save_pct")
        save1 = save[1].split('>')[1].split('<')[0]
        save2 = save[2].split('>')[1].split('<')[0]
        save_1 = float(save1) if save1 != '' else 0
        save_2 = float(save2) if save2 != '' else 0
     


        #collecting data
        date = table_3.get('data-venue-date')        


        #treating data
        match = str(soup.title)
        match = match.replace(" ","_")
        match = match.replace("<title>","")
        match = match.replace(".","")
        match_final = match.split("Report")[0]


        #treating data
        match_final = match_final.split("_Match")
        match_final = match_final[0]    


        # STR transform and reading tables
        table_str_1 = str(table_1)
        table_str_2 = str(table_2)
        df_1 = pd.read_html(table_str_1)[0]
        df_2 = pd.read_html(table_str_2)[0]

        #treating data
        team = str(match_final)
        team = team.replace("_"," ")
        team = team.split(" vs ")
        team_1 = str(team[0])
        team_2 = str(team[1])

        
        #Dtframe transforming
        df_1 = pd.DataFrame(df_1).tail(1)      #use only the summary row
        df_1.columns = df_1.columns.droplevel()
        df_1['Team'] = str(team_1)
        df_1['Home or Away'] = 'Home'
        df_1['Match'] = str(match_final)
        df_1['Date'] = str(date)
        df_1['Possession'] = possession_1
        df_1['Save%'] = save_1
  

        df_2 = pd.DataFrame(df_2).tail(1)      #use only the summary row
        df_2.columns = df_2.columns.droplevel()
        df_2['Team'] = str(team_2)
        df_2['Home or Away'] = 'Away'
        df_2['Match'] = str(match_final)
        df_2['Date'] = str(date)
        df_2['Possession'] = possession_2
        df_2['Save%'] = save_2

        
        df_all = df_all.append(df_1).append(df_2)
        
        ind += 1
        if ind%50 == 0:
            print('scraped %d matches'%ind)
    
    df_all.reset_index(drop=True, inplace=True)

In [5]:
%lprun -f get_df get_df(urls)

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches


Timer unit: 1e-07 s

Total time: 815.18 s
File: <ipython-input-4-415dae2f7cd4>
Function: get_df at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_df(urls):
     2                                               '''
     3                                               Description: This function goes to de URL of the match and treat all data in order to append it in one single Dataframe.
     4                                               
     5                                               Input:
     6                                                   - urls: urls of all the match reports
     7                                                   - league: league
     8                                                   - season: season
     9                                               Output:
    10                                                   - Dataframe treated from the match saved on my machine ex

In [71]:
# record the runtime for the original version
time_original = 815.18

## Advanced Python Techniques

### 1. Multi-threading

In [69]:
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import Pool

In [70]:
start = time.time()

with ThreadPoolExecutor(max_workers=8) as ex:
    ex.map(get_df, urls)

time_threading = time.time() - start
print('Time for scraping: {} seconds'.format(time_threading))

Time for scraping: 137.5474328994751 seconds


### 2. Cython with Predefined Ctype Variables

In [63]:
%load_ext Cython

In [64]:
from Cython.Compiler.Options import get_directive_defaults

directive_defaults = get_directive_defaults()
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [65]:
%%cython -f 
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

import pandas as pd
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import os
import time


def get_df_cython(urls):
    '''
    Description: This function goes to de URL of the match and treat all data in order to append it in one single Dataframe.
    
    Input:
        - urls: urls of all the match reports
        - league: league
        - season: season
    Output:
        - Dataframe treated from the match saved on my machine excel file
    
    '''
    df_all = pd.DataFrame()
    ind = 0
    
    # pre-define ctype variables
    cdef list percentage, save
    cdef str match, table_str_1, table_str_2, team_1, team_2
    cdef int possession_1, possession_2
    cdef float save_1, save_2


    
    for url in urls:    
        # make the request
        pg = 'https://fbref.com'
        url_pg = pg+ url
        req = requests.get(url_pg)
        if req.status_code == 200:
            content = req.content
        # accessing data from site
        soup = BeautifulSoup(content, 'html.parser')

        
        table_geral = soup.find_all(class_ = "table_wrapper tabbed")
        table_1 = table_geral[0]
        table_2 = table_geral[1]
        table_3 = soup.find(class_='venuetime')

        
        # possession
        percentage = str(soup.find_all('strong')).split('%')
        possession_1 = int(percentage[0][-2:])
        possession_2 = int(percentage[1][-2:])
        
        # save percentage
        save = str(soup.find_all('td')).split("save_pct")
        save1 = save[1].split('>')[1].split('<')[0]
        save2 = save[2].split('>')[1].split('<')[0]
        save_1 = float(save1) if save1 != '' else 0
        save_2 = float(save2) if save2 != '' else 0
     


        #collecting data
        date = table_3.get('data-venue-date')        


        #treating data
        match = str(soup.title)
        match = match.replace(" ","_")
        match = match.replace("<title>","")
        match = match.replace(".","")
        match_final = match.split("Report")[0]


        #treating data
        match_final = match_final.split("_Match")
        match_final = match_final[0]    


        # STR transform and reading tables
        table_str_1 = str(table_1)
        table_str_2 = str(table_2)
        df_1 = pd.read_html(table_str_1)[0]
        df_2 = pd.read_html(table_str_2)[0]

        #treating data
        team = str(match_final)
        team = team.replace("_"," ")
        team = team.split(" vs ")
        team_1 = str(team[0])
        team_2 = str(team[1])

        
        #Dtframe transforming
        df_1 = pd.DataFrame(df_1).tail(1)      #use only the summary row
        df_1.columns = df_1.columns.droplevel()
        df_1['Team'] = str(team_1)
        df_1['Home or Away'] = 'Home'
        df_1['Match'] = str(match_final)
        df_1['Date'] = str(date)
        df_1['Possession'] = possession_1
        df_1['Save%'] = save_1
  

        df_2 = pd.DataFrame(df_2).tail(1)      #use only the summary row
        df_2.columns = df_2.columns.droplevel()
        df_2['Team'] = str(team_2)
        df_2['Home or Away'] = 'Away'
        df_2['Match'] = str(match_final)
        df_2['Date'] = str(date)
        df_2['Possession'] = possession_2
        df_2['Save%'] = save_2

        
        df_all = df_all.append(df_1).append(df_2)
        
        ind += 1
        if ind%50 == 0:
            print('scraped %d matches'%ind)
        
    df_all.reset_index(drop=True, inplace=True)

In [68]:
start = time.time()

get_df_cython(urls)

time_cython = time.time() - start
print('Time for scraping: {} seconds'.format(time_cython))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scraping: 425.0426309108734 seconds


### 3. Parallel Computing with Numba

In [72]:
from __future__ import division
import numba
from numba import cuda, jit
import math
import time

In [73]:
@jit(parallel = True)
def get_df_numba(urls):
    '''
    Description: This function goes to de URL of the match and treat all data in order to append it in one single Dataframe.
    
    Input:
        - urls: urls of all the match reports
        - league: league
        - season: season
    Output:
        - Dataframe treated from the match saved on my machine excel file
    
    '''
    df_all = pd.DataFrame()
    ind = 0
    
    for url in urls:    
        # make the request
        pg = 'https://fbref.com'
        url_pg = pg+ url
        req = requests.get(url_pg)
        if req.status_code == 200:
            content = req.content
        # accessing data from site
        soup = BeautifulSoup(content, 'html.parser')

        
        table_geral = soup.find_all(class_ = "table_wrapper tabbed")
        table_1 = table_geral[0]
        table_2 = table_geral[1]
        table_3 = soup.find(class_='venuetime')

        
        # possession
        percentage = str(soup.find_all('strong')).split('%')
        possession_1 = int(percentage[0][-2:])
        possession_2 = int(percentage[1][-2:])
        
        # save percentage
        save = str(soup.find_all('td')).split("save_pct")
        save1 = save[1].split('>')[1].split('<')[0]
        save2 = save[2].split('>')[1].split('<')[0]
        save_1 = float(save1) if save1 != '' else 0
        save_2 = float(save2) if save2 != '' else 0
     


        #collecting data
        date = table_3.get('data-venue-date')        


        #treating data
        match = str(soup.title)
        match = match.replace(" ","_")
        match = match.replace("<title>","")
        match = match.replace(".","")
        match_final = match.split("Report")[0]


        #treating data
        match_final = match_final.split("_Match")
        match_final = match_final[0]    


        # STR transform and reading tables
        table_str_1 = str(table_1)
        table_str_2 = str(table_2)
        df_1 = pd.read_html(table_str_1)[0]
        df_2 = pd.read_html(table_str_2)[0]

        #treating data
        team = str(match_final)
        team = team.replace("_"," ")
        team = team.split(" vs ")
        team_1 = str(team[0])
        team_2 = str(team[1])

        
        #Dtframe transforming
        df_1 = pd.DataFrame(df_1).tail(1)      #use only the summary row
        df_1.columns = df_1.columns.droplevel()
        df_1['Team'] = str(team_1)
        df_1['Home or Away'] = 'Home'
        df_1['Match'] = str(match_final)
        df_1['Date'] = str(date)
        df_1['Possession'] = possession_1
        df_1['Save%'] = save_1
  

        df_2 = pd.DataFrame(df_2).tail(1)      #use only the summary row
        df_2.columns = df_2.columns.droplevel()
        df_2['Team'] = str(team_2)
        df_2['Home or Away'] = 'Away'
        df_2['Match'] = str(match_final)
        df_2['Date'] = str(date)
        df_2['Possession'] = possession_2
        df_2['Save%'] = save_2

        
        df_all = df_all.append(df_1).append(df_2)
        
        ind += 1
        if ind%50 == 0:
            print('scraped %d matches'%ind)
    
    df_all.reset_index(drop=True, inplace=True)

In [76]:
start = time.time()

get_df_numba(urls)

time_numba = time.time() - start
print('Time for scraping: {} seconds'.format(time_numba))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scraping: 364.4957365989685 seconds


## Accaceleration for Each Method

In [81]:
acc_threading = time_original/time_threading * 100
acc_cython = time_original/time_cython * 100
acc_numba = time_original/time_numba * 100

print(f'speed up for multi-threading: {int(acc_threading)} %')
print(f'speed up for cython: {int(acc_cython)} %')
print(f'speed up for numba: {int(acc_numba)} %')

speed up for multi-threading: 592 %
speed up for cython: 191 %
speed up for numba: 223 %
